In [1]:
import wandb
import os 

os.environ['WANDB_NOTEBOOK_NAME'] = 
%env WANDB_API_KEY = 

wandb.login()

SyntaxError: invalid syntax (2347968427.py, line 4)

In [2]:
import time
from torch.nn import BCEWithLogitsLoss
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
import gc  


INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.18 (you have 1.4.12). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


In [3]:
from utils import (
    save_checkpoint,
    load_checkpoint,
    check_accuracy,
    save_test_images,
    get_loaders,
    check_accuracy_val,
    check_accuracy_test
)

from models import UNET
from monai.networks.nets import UNet
from train import train_epoch
from simulated_lung_dataset_gaussian_std import get_loaders_simulated

In [8]:
# HYPERPARAMETERS:
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
PIN_MEMORY = False  # True if DEVICE == 'cuda' else False
LOGGING = True
BATCH_SIZE = 8
NUM_EPOCHS = 10
NUM_WORKERS = 0
IMAGE_SIZE = 256

# train

In [9]:
def model_train(model_input, save_model_path, LOGGING, IMAGE_STD, NOISE_STD_RATIO, DEVICE):    
    model = model_input
    loss_fn = BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scaler = torch.cuda.amp.GradScaler()

    # training
    best_loss = 1_000_000_000
    train_loader, test_loader, validation_loader = get_loaders_simulated(image_std = IMAGE_STD , noise_std_ratio= NOISE_STD_RATIO)
    #check_accuracy(0, validation_loader, model, logging = LOGGING ,device=DEVICE)
    for epoch in range(1, NUM_EPOCHS+1):
        loss = train_epoch(train_loader, model, optimizer, loss_fn, scaler, DEVICE)
        if epoch % 5 == 0:
            check_accuracy_val(epoch, validation_loader, model, logging = LOGGING ,device=DEVICE)
        del loss
        torch.cuda.empty_cache() 
        
    checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
            }
    check_accuracy_test(epoch, test_loader, model, logging = LOGGING ,device=DEVICE)
    save_checkpoint(checkpoint, filename= save_model_path)

# main

In [10]:
def model_train_log(i, model, model_name, LOGGING, IMAGE_STD, NOISE_STD_RATIO,  DEVICE = 'cuda'):
    model_save_path = f'/home/alex/Documents/new try/Data/Lung Unet/save_states/Monai_Unet_gaussian_test_{IMAGE_STD}_{NOISE_STD_RATIO}.pth.tar' 
    
    #logging
    if LOGGING:
        run = wandb.init(
            # set the wandb project where this run will be logged
            #entity = "alexander-ortlieb94",
            project = "Compare continued simulations monai",
            group = "Simulated",
            job_type = 'Gaussian',
            name = 'Monai',

            # track hyperparameters and run metadata
            config={
                "try no." : i,
                "std_matching" : IMAGE_STD,
                "std_noise_ratio" : NOISE_STD_RATIO
                },
        )
    else:
        pass
 
    
    model_train(
        model,
        model_save_path,
        LOGGING,
        IMAGE_STD, 
        NOISE_STD_RATIO, 
        DEVICE)
    
    if LOGGING:
        run.finish()

    torch.cuda.empty_cache() 
   

In [11]:
def run_tests():

    for i in range(3,4):
        for IMAGE_STD in [10]: #[3, 5, 7, 10]:
            for NOISE_STD_RATIO in [0.1]:
                LOGGING = True
                #Own_unet = UNET(in_channels=1,out_channels=1, features= [16, 32, 64, 128]).to(DEVICE)
                monai_UNet = UNet(spatial_dims=2, in_channels=1, out_channels=1, channels=(16, 32, 64, 128), strides=(2, 2, 2), bias = False, num_res_units = 0).to(device=DEVICE)
                load_checkpoint(monai_UNet, r'/home/alex/Documents/new try/Data/Lung Unet/save_states/simulate_data/Monai_Unet_gaussian1.pth.tar')
                model_train_log(i, monai_UNet, 'Own_unet', LOGGING, IMAGE_STD, NOISE_STD_RATIO, DEVICE)
                

if __name__ == "__main__":
    run_tests()

=> Loading checkpoint


100%|██████████| 62/62 [02:12<00:00,  2.14s/it, loss=0.211]


Val-Epoch: 5, Acc: 0.90, and Dice score: 0.65, IoU: 0.68, hd: 32.60


100%|██████████| 62/62 [02:11<00:00,  2.12s/it, loss=0.174]


Val-Epoch: 10, Acc: 0.90, and Dice score: 0.66, IoU: 0.70, hd: 32.17
Epoch: 10, Acc: 0.90, and Dice score: 0.69, IoU: 0.76, hd: 25.59
=> Saving checkpoint


epoch,▁
test_acc,▁
test_box_iou,▁
test_dice,▁
test_hd,▁
test_loss,▁
epoch,10
test_acc,0.90211
test_box_iou,0.76313
test_dice,0.69419
test_hd,25.59276
